In [3]:
!pip install fastapi uvicorn

In [1]:
import subprocess, sys

server = subprocess.Popen([
    sys.executable, "-m", "uvicorn", "main:app",
    "--host", "127.0.0.1", "--port", "8000"
])

print("Servidor iniciado en http://127.0.0.1:8000")

Servidor iniciado en http://127.0.0.1:8000


In [5]:
#status de conexion
import requests
requests.get("http://127.0.0.1:8000/health").json()


{'Status': 'Ok'}

In [ ]:
!pip install xgboost

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Ruta del CSV (debe estar en el mismo directorio que este notebook)
CSV_PATH = 'Fleet_Work_Orders_20251111.csv'
assert Path(CSV_PATH).exists(), f"No encontré {CSV_PATH} en el directorio actual."

# Cargar y mostrar primeras filas
df = pd.read_csv(CSV_PATH)
print(f"Dataset original: {df.shape}")

C:\Users\thebe\AppData\Local\Temp\ipykernel_23448\2442315454.py:12: DtypeWarning: Columns (18,19,20,21,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(CSV_PATH)


Dataset original: (472112, 38)


# Bloque de limpieza

In [9]:
# 1. CONVERSIÓN DE FECHAS (igual que las que teniamos Omar saldivar, che corrupto!)
date_cols = [
    'In Service Date',
    'Work Order Begin Date', 
    'Work Order Finish Date'
]
for c in date_cols:
    df[c] = pd.to_datetime(df[c], errors='coerce')

In [11]:
# 2. FILTRAR SOLO MANTENIMIENTOS NORMALES (CRÍTICO)
print(f"Registros antes de filtrar: {len(df)}")
df = df[df['WO Reason'].str.contains('REPAIR/MAINTENANCE', na=False)].copy()
print(f"Registros después de filtrar solo mantenimientos: {len(df)}")

Registros antes de filtrar: 472112
Registros después de filtrar solo mantenimientos: 301891


In [13]:
# 3. LIMPIAR TEXTOS (igual a como teniamos)
text_cols = ['Vehicle Make', 'Vehicle Model', 'WO Reason']
for c in text_cols:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip().str.upper().replace({'NAN':'UNKNOWN'})

In [15]:
# 4. NORMALIZAR INDICADORES (igual que los que teníamos)
if 'On Time Indicator' in df.columns:
    df['On Time Indicator'] = df['On Time Indicator'].astype(str).str.upper().str.strip()
    df['On Time Indicator'] = df['On Time Indicator'].replace({'YES':'Y','NO':'N','1':'Y','0':'N'})
    df['On Time Indicator'] = df['On Time Indicator'].replace({'NAN':'UNKNOWN'})

if 'Open Indicator' in df.columns:
    df['Open Indicator'] = df['Open Indicator'].astype(str).str.upper().str.strip()
    df['Open Indicator'] = df['Open Indicator'].replace({'YES':'Y','NO':'N','1':'Y','0':'N'})
    df['Open Indicator'] = df['Open Indicator'].replace({'NAN':'UNKNOWN'})

In [17]:
# 5. FORZAR Y LIMPIAR NUMÉRICOS (MEJORADO)
numeric_fix_cols = [
    'Total Cost',
    'Expected Hours', 
    'Actual Hours',
    'Days to Complete',
    'WO Vehicle Odometer',
    'Vehicle Year'
]

for col in numeric_fix_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace(',', '', regex=False).str.replace('$','',regex=False).str.strip()
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [21]:
# 6. IMPUTACIÓN INTELIGENTE (NO CEROS)
print("\n=== IMPUTACIÓN DE VALORES FALTANTES ===")
for col in numeric_fix_cols:
    if col in df.columns:
        antes = df[col].isna().sum()
        
        if col == 'WO Vehicle Odometer':
            # Para odómetro: mediana por clase de vehículo
            df[col] = df.groupby('Vehicle Class')[col].transform(lambda x: x.fillna(x.median()))
        elif col == 'Vehicle Year':
            # Para año: moda por modelo
            df[col] = df.groupby('Vehicle Model')[col].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x.median()))
        else:
            # Para costos y horas: mediana general
            df[col] = df[col].fillna(df[col].median())
        
        despues = df[col].isna().sum()
        print(f"{col}: {antes} → {despues} valores faltantes")


=== IMPUTACIÓN DE VALORES FALTANTES ===
Total Cost: 0 → 0 valores faltantes
Expected Hours: 0 → 0 valores faltantes
Actual Hours: 0 → 0 valores faltantes
Days to Complete: 0 → 0 valores faltantes
WO Vehicle Odometer: 16176 → 16176 valores faltantes
Vehicle Year: 0 → 0 valores faltantes


In [23]:
# 7. ORDENAR POR VEHÍCULO Y FECHA (CRÍTICO PARA SERIES TEMPORALES)
df = df.sort_values(['Vehicle Number', 'Work Order Begin Date']).reset_index(drop=True)

In [29]:
print("\n=== CALCULANDO VARIABLES TEMPORALES ===")

# Calcular diferencia normal entre mantenimientos consecutivos
df['Dias_desde_ultimo_mantenimiento'] = df.groupby('Vehicle Number')['Work Order Begin Date'].diff().dt.days

# Obtener los índices REALES del DataFrame para los primeros mantenimientos
primeros_mantenimientos = df.groupby('Vehicle Number')['Work Order Begin Date'].idxmin()

# Convertir la Series de índices a una lista
primeros_mantenimientos_indices = primeros_mantenimientos.tolist()

print(f"Primeros mantenimientos identificados: {len(primeros_mantenimientos_indices)}")
print(f"Ejemplo de índices: {primeros_mantenimientos_indices[:5]}")

# Calcular días desde fecha de servicio para los primeros mantenimientos
df.loc[primeros_mantenimientos_indices, 'Dias_desde_ultimo_mantenimiento'] = (
    df.loc[primeros_mantenimientos_indices, 'Work Order Begin Date'] - 
    df.loc[primeros_mantenimientos_indices, 'In Service Date']
).dt.days

print(f"Registros con días desde último mantenimiento calculado: {df['Dias_desde_ultimo_mantenimiento'].notna().sum()}")
print(f"Valores nulos restantes: {df['Dias_desde_ultimo_mantenimiento'].isna().sum()}")

# Verificar distribución
print(f"\n DISTRIBUCIÓN DE 'Dias_desde_ultimo_mantenimiento':")
print(df['Dias_desde_ultimo_mantenimiento'].describe())

# Verificar los vehículos con solo 1 registro
vehiculos_un_registro = df.groupby('Vehicle Number').size()
vehiculos_un_registro = vehiculos_un_registro[vehiculos_un_registro == 1]
print(f"\n Vehículos con solo 1 registro: {len(vehiculos_un_registro)}")


=== CALCULANDO VARIABLES TEMPORALES ===
Primeros mantenimientos identificados: 4185
Ejemplo de índices: [5, 119, 166, 187, 282]
Registros con días desde último mantenimiento calculado: 272412
Valores nulos restantes: 0

 DISTRIBUCIÓN DE 'Dias_desde_ultimo_mantenimiento':
count    272412.000000
mean         59.098979
std         304.173305
min           0.000000
25%           3.000000
50%          12.000000
75%          48.000000
max       16545.000000
Name: Dias_desde_ultimo_mantenimiento, dtype: float64

 Vehículos con solo 1 registro: 195


C:\Users\thebe\AppData\Local\Temp\ipykernel_23448\1846964059.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Dias_desde_ultimo_mantenimiento'] = df.groupby('Vehicle Number')['Work Order Begin Date'].diff().dt.days


In [27]:
# 9. VERIFICAR INTEGRIDAD TEMPORAL
fechas_inconsistentes = (
    (df['Work Order Begin Date'] > df['Work Order Finish Date']) |
    (df['In Service Date'] > df['Work Order Begin Date']) |
    (df['Dias_desde_ultimo_mantenimiento'] < 0)
).sum()

print(f"Registros con fechas inconsistentes: {fechas_inconsistentes}")

# Eliminar registros temporalmente imposibles
df = df[
    (df['Work Order Begin Date'] <= df['Work Order Finish Date']) &
    (df['In Service Date'] <= df['Work Order Begin Date']) & 
    (df['Dias_desde_ultimo_mantenimiento'] >= 0)
]

Registros con fechas inconsistentes: 1066


In [31]:
# 10. ELIMINAR DUPLICADOS (igual que tenías)
df = df.drop_duplicates()
print(f"\nDataset final limpio: {df.shape}")


Dataset final limpio: (272412, 39)


In [33]:
# 11. MOSTRAR RESUMEN FINAL
print("\n=== RESUMEN FINAL ===")
print(f"Vehículos únicos: {df['Vehicle Number'].nunique()}")
print(f"Rango de fechas: {df['Work Order Begin Date'].min()} a {df['Work Order Begin Date'].max()}")
print(f"Días entre mantenimientos - Media: {df['Dias_desde_ultimo_mantenimiento'].mean():.1f}, " 
      f"Mediana: {df['Dias_desde_ultimo_mantenimiento'].median():.1f}")


=== RESUMEN FINAL ===
Vehículos únicos: 4185
Rango de fechas: 1997-09-04 00:00:00 a 2025-11-14 00:00:00
Días entre mantenimientos - Media: 59.1, Mediana: 12.0


In [35]:
# Mostrar tipos de datos finales
print("\nTipos de datos finales:")
df[numeric_fix_cols + ['Dias_desde_ultimo_mantenimiento']].info()


Tipos de datos finales:
<class 'pandas.core.frame.DataFrame'>
Index: 272412 entries, 5 to 301889
Data columns (total 7 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Total Cost                       272412 non-null  float64
 1   Expected Hours                   272412 non-null  float64
 2   Actual Hours                     272412 non-null  float64
 3   Days to Complete                 272412 non-null  float64
 4   WO Vehicle Odometer              272412 non-null  float64
 5   Vehicle Year                     272412 non-null  float64
 6   Dias_desde_ultimo_mantenimiento  272412 non-null  float64
dtypes: float64(7)
memory usage: 24.7 MB


# Targets y features section

In [37]:
# === TARGETS Y FEATURES CORREGIDOS ===

print("=== PREPARANDO TARGETS Y FEATURES ===")

# 1. CALCULAR TARGET PRINCIPAL: Días hasta próximo mantenimiento
df = df.sort_values(['Vehicle Number', 'Work Order Begin Date'])

# Calcular próximo mantenimiento
df['next_begin'] = df.groupby('Vehicle Number')['Work Order Begin Date'].shift(-1)
df['days_to_next_maintenance'] = (df['next_begin'] - df['Work Order Finish Date']).dt.days

# Identificar y excluir últimos mantenimientos de cada vehículo (donde no hay próximo)
ultimos_mantenimientos = df.groupby('Vehicle Number')['Work Order Begin Date'].idxmax()
print(f"Últimos mantenimientos excluidos: {len(ultimos_mantenimientos)}")

# Crear dataset de entrenamiento (excluyendo últimos mantenimientos)
df_train = df[~df.index.isin(ultimos_mantenimientos)].copy()
df_train = df_train[df_train['days_to_next_maintenance'] > 0]  # Excluir valores negativos/inválidos

print(f"Registros para entrenamiento: {len(df_train)}")
print(f"Rango del target: {df_train['days_to_next_maintenance'].min()} a {df_train['days_to_next_maintenance'].max()} días")

=== PREPARANDO TARGETS Y FEATURES ===
Últimos mantenimientos excluidos: 4185
Registros para entrenamiento: 226921
Rango del target: 1.0 a 8022.0 días


In [39]:
# 2. FEATURES CORRECTAS (SOLO INFORMACIÓN DEL PASADO/PRESENTE)
feature_cols_corrected = [
    # === HISTORIAL DEL VEHÍCULO (INFORMACIÓN DISPONIBLE) ===
    'Dias_desde_ultimo_mantenimiento',  #Días desde el último mantenimiento
    'WO Vehicle Odometer',              #Kilometraje actual (al momento del WO)
    'Vehicle Year',                     #Año del vehículo
    'Vehicle Make',                     #Marca
    'Vehicle Model',                    #Modelo Y NO MISS UNIVERSO, NETA SII!!  
    'Vehicle Class',                    #Clase (H, L, M, O)
    'Fuel Type',                        #Tipo combustible
    'Division Owner',                   #Departamento/área
    
    # === CARACTERÍSTICAS DEL MANTENIMIENTO ACTUAL ===
    'Expected Hours',                   #Horas estimadas (se saben al inicio)
    'WO Reason',                        #Razón del mantenimiento
    'Priority Code',                    #Prioridad (1, 2, 3...)
    'Shop Name',                        #Taller donde se realiza
    
    # === INFORMACIÓN TEMPORAL/DEMOGRÁFICA ===
    'Days to Complete',                 #Tiempo que tomó el ÚLTIMO mantenimiento
    'Invoice Month Year',               #Mes/año (para patrones estacionales)
]

# Filtrar solo las columnas que existen en el dataset
feature_cols_corrected = [c for c in feature_cols_corrected if c in df_train.columns]
print(f"\nFeatures seleccionados: {len(feature_cols_corrected)}")
print(feature_cols_corrected)


Features seleccionados: 14
['Dias_desde_ultimo_mantenimiento', 'WO Vehicle Odometer', 'Vehicle Year', 'Vehicle Make', 'Vehicle Model', 'Vehicle Class', 'Fuel Type', 'Division Owner', 'Expected Hours', 'WO Reason', 'Priority Code', 'Shop Name', 'Days to Complete', 'Invoice Month Year']


In [41]:
# 3. PREPARAR DATOS PARA ENTRENAMIENTO
X = df_train[feature_cols_corrected].copy()
y_next = df_train['days_to_next_maintenance']  # Target principal

print(f"\nShape de X: {X.shape}")
print(f"Shape de y: {y_next.shape}")


Shape de X: (226921, 14)
Shape de y: (226921,)


In [43]:
# 4. PREPROCESAMIENTO INTELIGENTE DE FEATURES
print("\n=== PREPROCESAMIENTO DE FEATURES ===")

# Identificar tipos de columnas
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Columnas numéricas ({len(num_cols)}): {num_cols}")
print(f"Columnas categóricas ({len(cat_cols)}): {cat_cols}")

# A. Limpiar y normalizar categóricas
for col in cat_cols:
    X[col] = X[col].astype(str).str.upper().str.strip()
    X[col] = X[col].replace({'NAN': 'UNKNOWN', 'NONE': 'UNKNOWN', '': 'UNKNOWN'})
    X[col] = X[col].astype('category')
    # Asegurar que 'UNKNOWN' está en las categorías
    if 'UNKNOWN' not in X[col].cat.categories:
        X[col] = X[col].cat.add_categories(['UNKNOWN'])

# B. Manejo de valores faltantes
print("\n=== MANEJO DE VALORES FALTANTES ===")
for col in X.columns:
    if X[col].isna().sum() > 0:
        if col in num_cols:
            # Para numéricos: mediana por grupo relevante
            if col == 'WO Vehicle Odometer':
                X[col] = X.groupby('Vehicle Class')[col].transform(lambda x: x.fillna(x.median()))
            else:
                X[col] = X[col].fillna(X[col].median())
        else:
            # Para categóricas: 'UNKNOWN'
            X[col] = X[col].fillna('UNKNOWN')
        print(f" {col}: {X[col].isna().sum()} faltantes después de imputación")

# C. Crear features adicionales derivadas
print("\n=== CREANDO FEATURES ADICIONALES ===")

# Edad del vehículo al momento del mantenimiento
if 'Vehicle Year' in X.columns and 'Work Order Begin Date' in df_train.columns:
    X['vehicle_age_years'] = df_train['Work Order Begin Date'].dt.year - X['Vehicle Year']
    num_cols.append('vehicle_age_years')

# Mes y día de la semana para patrones estacionales
if 'Work Order Begin Date' in df_train.columns:
    X['work_order_month'] = df_train['Work Order Begin Date'].dt.month
    X['work_order_dayofweek'] = df_train['Work Order Begin Date'].dt.dayofweek
    num_cols.extend(['work_order_month', 'work_order_dayofweek'])

# Extraer año de Invoice Month Year si existe
if 'Invoice Month Year' in X.columns:
    try:
        X['invoice_year'] = X['Invoice Month Year'].astype(str).str[:2].astype(float) + 2000
        num_cols.append('invoice_year')
    except:
        print("No se pudo extraer año de Invoice Month Year")

print(f"Features finales: {len(X.columns)}")
print(f"Columnas numéricas: {len(num_cols)}")
print(f"Columnas categóricas: {len(cat_cols)}")


=== PREPROCESAMIENTO DE FEATURES ===
Columnas numéricas (7): ['Dias_desde_ultimo_mantenimiento', 'WO Vehicle Odometer', 'Vehicle Year', 'Expected Hours', 'Priority Code', 'Days to Complete', 'Invoice Month Year']
Columnas categóricas (7): ['Vehicle Make', 'Vehicle Model', 'Vehicle Class', 'Fuel Type', 'Division Owner', 'WO Reason', 'Shop Name']

=== MANEJO DE VALORES FALTANTES ===
 Invoice Month Year: 0 faltantes después de imputación

=== CREANDO FEATURES ADICIONALES ===
Features finales: 18
Columnas numéricas: 11
Columnas categóricas: 7


In [45]:
# 5. VERIFICACIÓN FINAL DE INTEGRIDAD
print("\n=== VERIFICACIÓN FINAL ===")
print(f" X shape: {X.shape}")
print(f" y shape: {y_next.shape}")
print(f" Valores faltantes en X: {X.isna().sum().sum()}")
print(f" Valores faltantes en y: {y_next.isna().sum()}")
print(f" Rango del target: {y_next.min()} - {y_next.max()} días")

# Distribución del target
print(f"\n DISTRIBUCIÓN DEL TARGET 'days_to_next_maintenance':")
print(y_next.describe())

# Verificar que no hay fugas de datos
leakage_check = ['Total Cost', 'Actual Hours', 'On Time Indicator']
for col in leakage_check:
    if col in X.columns:
        print(f" ALERTA: {col} es FUGA DE DATOS - ELIMINAR!")

print("\nEN TEORIA YA ESTA LISTO PARA ENTRENAMIENTO")


=== VERIFICACIÓN FINAL ===
 X shape: (226921, 18)
 y shape: (226921,)
 Valores faltantes en X: 0
 Valores faltantes en y: 0
 Rango del target: 1.0 - 8022.0 días

 DISTRIBUCIÓN DEL TARGET 'days_to_next_maintenance':
count    226921.000000
mean         48.515664
std         125.078486
min           1.000000
25%           4.000000
50%          16.000000
75%          54.000000
max        8022.000000
Name: days_to_next_maintenance, dtype: float64

EN TEORIA YA ESTA LISTO PARA ENTRENAMIENTO


 # SECCION DE PARA EL MODELADO

In [47]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor, XGBClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, f1_score, classification_report
import numpy as np
def train_and_eval_reg(X, y, label='reg'):
    """
    Versión ENHANCEDC COMO DICE LA CHAVIZA validación de datos y manejo de categóricas
    """
    print(f"\n ENTRENANDO MODELO: {label}")
    print(f"   Samples: {len(X)}, Features: {X.shape[1]}")
    print(f"   Target range: {y.min():.1f} to {y.max():.1f}")
    
    # Verificar que hay suficientes datos
    if len(X) < 100:
        print(f" MUY POCOS DATOS: {len(X)} muestras")
        return None, None, None, None
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, 
        stratify=X['Vehicle Class'] if 'Vehicle Class' in X.columns else None
    )
    
    # Preparar datos para XGBoost (codificar categóricas)
    X_train_encoded = X_train.copy()
    X_test_encoded = X_test.copy()
    
    cat_cols = X_train.select_dtypes(include=['category', 'object']).columns
    for col in cat_cols:
        # Codificación por frecuencia (mejor que one-hot para XGBoost)
        X_train_encoded[col] = X_train[col].astype('category').cat.codes
        X_test_encoded[col] = X_test[col].astype('category').cat.codes
    
    model = XGBRegressor(
        n_estimators=200, 
        learning_rate=0.05, 
        max_depth=6,
        random_state=42,
        enable_categorical=False  # Ya codificamos manualmente
    )
    
    model.fit(X_train_encoded, y_train)
    preds = model.predict(X_test_encoded)
    
    mae = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    
    print(f" {label} MAE: {mae:.2f} | RMSE: {rmse:.2f}")
    print(f" Error relativo: {(mae / y.mean() * 100):.1f}% del promedio")
    
    return model, X_test_encoded, y_test, preds
def train_and_eval_clf(X, y, label='clf'):
    """
    clasificación
    """
    print(f"\n ENTRENANDO CLASIFICADOR: {label}")
    
    # Convertir target a 0/1
    y_clean = y.astype(str).str.upper().map({'Y': 1, 'N': 0, 'YES': 1, 'NO': 0})
    y_clean = y_clean.fillna(0).astype(int)
    
    # Verificar balance de clases
    class_counts = y_clean.value_counts()
    print(f"   Distribución clases: {dict(class_counts)}")
    
    if len(class_counts) < 2:
        print(" SOLO UNA CLASE - No se puede entrenar clasificador")
        return None, None, None, None
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_clean, test_size=0.2, random_state=42,
        stratify=y_clean
    )
    
    # Codificar categóricas
    X_train_encoded = X_train.copy()
    X_test_encoded = X_test.copy()
    
    cat_cols = X_train.select_dtypes(include=['category', 'object']).columns
    for col in cat_cols:
        X_train_encoded[col] = X_train[col].astype('category').cat.codes
        X_test_encoded[col] = X_test[col].astype('category').cat.codes
    
    model = XGBClassifier(
        n_estimators=200, 
        max_depth=6, 
        random_state=42,
        enable_categorical=False
    )
    
    model.fit(X_train_encoded, y_train)
    preds = model.predict(X_test_encoded)
    
    acc = accuracy_score(y_test, preds)
    f1 = f1_score(y_test, preds, zero_division=0)
    
    print(f"  {label} ACC: {acc:.4f} | F1: {f1:.4f}")
    print(classification_report(y_test, preds, zero_division=0))
    
    return model, X_test_encoded, y_test, preds

# === ENTRENAMIENTO CON FEATURES CORREGIDAS ===

In [49]:
print("=" * 60)
print(" COMIENZA ENTRENAMIENTO CON FEATURES CORREGIDAS")
print("=" * 60)

# 1) Modelo PRINCIPAL: Days to next maintenance (REGRESIÓN)
print("\n" + "="*50)
print(" MODELO PRINCIPAL: Días hasta próximo mantenimiento")
print("="*50)

model_next, Xn_test, yn_test, preds_next = train_and_eval_reg(
    X, y_next, label='DaysToNextMaintenance'
)

 COMIENZA ENTRENAMIENTO CON FEATURES CORREGIDAS

 MODELO PRINCIPAL: Días hasta próximo mantenimiento

 ENTRENANDO MODELO: DaysToNextMaintenance
   Samples: 226921, Features: 18
   Target range: 1.0 to 8022.0
 DaysToNextMaintenance MAE: 40.08 | RMSE: 111.56
 Error relativo: 82.6% del promedio


In [51]:
# 2) Modelo para Costo (REGRESIÓN) - CON FEATURES APROPIADAS
print("\n" + "="*50)
print(" MODELO: Costo de mantenimiento")
print("="*50)

# Para costo, usar features diferentes (sin información futura)
cost_features = [
    'Vehicle Year', 'Vehicle Make', 'Vehicle Model', 'Vehicle Class',
    'Fuel Type', 'Division Owner', 'WO Reason', 'Priority Code',
    'Expected Hours', 'Shop Name', 'Dias_desde_ultimo_mantenimiento',
    'WO Vehicle Odometer', 'work_order_month', 'vehicle_age_years'
]

# Filtrar solo las que existen en X (que ya tiene todas las features derivadas)
cost_features = [f for f in cost_features if f in X.columns]
print(f"   Features de costo disponibles: {cost_features}")

if cost_features and 'Total Cost' in df_train.columns:
    # Usar X que ya tiene todas las features preparadas
    X_cost = X[cost_features].copy()
    y_cost = df_train['Total Cost']
    
    # Limpiar outliers extremos de costo
    cost_q99 = y_cost.quantile(0.99)
    mask_cost = y_cost <= cost_q99
    X_cost_clean = X_cost[mask_cost].copy()
    y_cost_clean = y_cost[mask_cost]
    
    print(f"   Costo - Datos después de limpiar outliers: {len(X_cost_clean)}")
    
    model_cost, Xc_test, yc_test, preds_cost = train_and_eval_reg(
        X_cost_clean, y_cost_clean, label='TotalCost'
    )
else:
    print("  No se pueden entrenar modelo de costo - features faltantes")
    model_cost = None


 MODELO: Costo de mantenimiento
   Features de costo disponibles: ['Vehicle Year', 'Vehicle Make', 'Vehicle Model', 'Vehicle Class', 'Fuel Type', 'Division Owner', 'WO Reason', 'Priority Code', 'Expected Hours', 'Shop Name', 'Dias_desde_ultimo_mantenimiento', 'WO Vehicle Odometer', 'work_order_month', 'vehicle_age_years']
   Costo - Datos después de limpiar outliers: 224651

 ENTRENANDO MODELO: TotalCost
   Samples: 224651, Features: 14
   Target range: -21305.7 to 3917.5
 TotalCost MAE: 178.82 | RMSE: 364.60
 Error relativo: 60.7% del promedio


In [53]:
# 3) Modelo para Tiempo de ejecución (REGRESIÓN)
print("\n" + "="*50)
print(" MODELO: Horas de trabajo")
print("="*50)

time_features = [
    'Vehicle Make', 'Vehicle Model', 'Vehicle Class', 'WO Reason',
    'Priority Code', 'Expected Hours', 'Shop Name', 'work_order_month'
]
time_features = [f for f in time_features if f in X.columns]  # ← Cambiar a X.columns

if time_features and 'Actual Hours' in df_train.columns:
    X_time = X[time_features].copy()  # ← Usar X en lugar de df_train
    y_time = df_train['Actual Hours']
    
     # Limpiar outliers
    time_q99 = y_time.quantile(0.99)
    mask_time = y_time <= time_q99
    X_time_clean = X_time[mask_time].copy()
    y_time_clean = y_time[mask_time]
    
    model_time, Xt_test, yt_test, preds_time = train_and_eval_reg(
        X_time_clean, y_time_clean, label='ActualHours'
    )
else:
    print(" No se pueden entrenar modelo de tiempo - features faltantes")
    model_time = None


 MODELO: Horas de trabajo

 ENTRENANDO MODELO: ActualHours
   Samples: 224654, Features: 8
   Target range: -43.0 to 20.2
 ActualHours MAE: 0.68 | RMSE: 1.45
 Error relativo: 35.2% del promedio


In [55]:
# 4) Clasificador para Cumplimiento de tiempo (CLASIFICACIÓN)
print("\n" + "="*50)
print(" CLASIFICADOR: Indicador de cumplimiento")
print("="*50)

if 'On Time Indicator' in df_train.columns:
    ontime_features = [
        'Vehicle Make', 'Vehicle Model', 'Shop Name', 'Priority Code',
        'Expected Hours', 'work_order_month', 'work_order_dayofweek',
        'Division Owner', 'Vehicle Class'
    ]
    ontime_features = [f for f in ontime_features if f in X.columns]  # ← X.columns
    
    if ontime_features:
        X_ontime = X[ontime_features].copy()  # ← Usar X
        y_ontime = df_train['On Time Indicator']

        model_on_time, Xot_test, yot_test, preds_ot = train_and_eval_clf(  # ← SIN ESPACIO EXTRA
            X_ontime, y_ontime, label='OnTime'
        )
    else:
        print(" No hay features para modelo de cumplimiento")
        model_on_time = None
else:
    print(" No se encontró 'On Time Indicator' en los datos")
    model_on_time = None


 CLASIFICADOR: Indicador de cumplimiento

 ENTRENANDO CLASIFICADOR: OnTime
   Distribución clases: {1: 156887, 0: 70034}
  OnTime ACC: 0.7811 | F1: 0.8519
              precision    recall  f1-score   support

           0       0.71      0.49      0.58     14007
           1       0.80      0.91      0.85     31378

    accuracy                           0.78     45385
   macro avg       0.76      0.70      0.72     45385
weighted avg       0.77      0.78      0.77     45385



In [57]:
# === GUARDAR MODELOS ===
print("\n" + "="*50)
print("GUARDANDO MODELOS ENTRENADOS")
print("="*50)

models_guardados = []

if model_next is not None:
    model_next.save_model('model_next_maintenance.json')
    models_guardados.append('model_next_maintenance.json')
    print("Modelo principal (próximo mantenimiento) guardado")

if model_cost is not None:
    model_cost.save_model('model_cost.json')
    models_guardados.append('model_cost.json')
    print("Modelo de costo guardado")

if model_time is not None:
    model_time.save_model('model_time.json') 
    models_guardados.append('model_time.json')
    print("Modelo de tiempo guardado")

if model_on_time is not None:
    model_on_time.save_model('model_on_time.json')
    models_guardados.append('model_on_time.json')
    print("Modelo de cumplimiento guardado")

print(f"\n ENTRENAMIENTO COMPLETADO: {len(models_guardados)} modelos guardados")
print("Archivos:", ", ".join(models_guardados))



GUARDANDO MODELOS ENTRENADOS
Modelo principal (próximo mantenimiento) guardado
Modelo de costo guardado
Modelo de tiempo guardado
Modelo de cumplimiento guardado

 ENTRENAMIENTO COMPLETADO: 4 modelos guardados
Archivos: model_next_maintenance.json, model_cost.json, model_time.json, model_on_time.json


In [59]:
from xgboost import XGBRegressor, XGBClassifier

# Cargar los modelos
model_cost = XGBRegressor()
model_cost.load_model("model_cost.json")

model_time = XGBRegressor()
model_time.load_model("model_time.json")

model_next = XGBRegressor()
model_next.load_model("model_next_maintenance.json")

# Clasificador (solo si existe)
model_on_time = XGBClassifier()
model_on_time.load_model("model_on_time.json")

print("Modelos cargados exitosamente")


Modelos cargados exitosamente


## Pruebas 

In [61]:
#Crear un diccionario de entrada básico
#definimos nuestros datos de prueba 
mi_input = {
    "Vehicle Year": 2019,
    "Vehicle Make": "FORD",
    "Vehicle Model": "F150",
    "Vehicle Class": "TRUCK",
    "Fuel Type": "GAS",
    "Division Owner": "CITY",
    "WO Reason": "REPAIR/MAINTENANCE",
    "Priority Code": 2,
    "Expected Hours": 5,
    "Shop Name": "MAIN GARAGE",
    "Dias_desde_ultimo_mantenimiento": 120,
    "WO Vehicle Odometer": 45000,
    "Days to Complete": 3,
    "Invoice Month Year": 202402,
    "work_order_month": 5,
    "work_order_dayofweek": 3,
    "invoice_year": 2024
}

In [63]:
#Convertirlo a DataFrame
df_input = pd.DataFrame([mi_input]) # para que pandas y XGBoost lo manejen simultaneamente

In [65]:
#Convertir categóricas → category para xgboost para evitar errores de tipo
for col in cat_cols:  
    df_input[col] = df_input[col].astype(str).str.upper().astype("category")

In [73]:
pred = model_cost.predict(X_cost)[0] #analizamos el modelo de costo

print("===== RESULTADO DEL MODELO DE COSTO =====")
print(f" Vehículo: {mi_input['Vehicle Make']} {mi_input['Vehicle Model']} {mi_input['Vehicle Year']}")
print(f" Tipo de mantenimiento: {mi_input['WO Reason']}")
print(f" Costo estimado: ${pred:,.2f} USD")
print("=========================================")


===== RESULTADO DEL MODELO DE COSTO =====
 Vehículo: FORD F150 2019
 Tipo de mantenimiento: PREVENTIVE
 Costo estimado: $230.46 USD


In [69]:
def preparar_input_tiempo(datos): #analizamos el modelo de horas de trabajo
    """
    Predecir el tiempo real del mantenimiento (Actual Hours)
    basado en las mismas transformaciones que se aplicaron a X_time.
    """

    # 1. Convertir a DataFrame
    df_in = pd.DataFrame([datos])

    # 2. Asegurar que las columnas necesarias existen
    for col in X_time.columns:
        if col not in df_in.columns:
            df_in[col] = np.nan

    # 3. Procesar categóricas tal como se hizo en el entrenamiento
    for col in X_time.select_dtypes(include=['category']).columns:
        df_in[col] = df_in[col].astype(str).str.upper().str.strip()

        # Si no existe en las categorías del modelo, marcar como UNKNOWN
        categorias = X_time[col].cat.categories
        df_in[col] = df_in[col].apply(lambda x: x if x in categorias else 'UNKNOWN')

        df_in[col] = pd.Categorical(df_in[col], categories=categorias)

    # 4. Rellenar numéricas con mediana
    for col in X_time.select_dtypes(include=['int64','float64']).columns:
        if df_in[col].isna().any():
            df_in[col] = df_in[col].fillna(X_time[col].median())

    return df_in[X_time.columns]

# ==========================
#   EJEMPLO DE PREDICCIÓN
# ==========================

mi_input = {
    "Vehicle Year": 2019,
    "Vehicle Make": "FORD",
    "Vehicle Model": "F150",
    "Vehicle Class": "TRUCK",
    "Fuel Type": "GAS",
    "Division Owner": "CITY",
    "WO Reason": "PREVENTIVE",
    "Priority Code": 2,
    "Expected Hours": 4,
    "Shop Name": "MAIN GARAGE",
    "Dias_desde_ultimo_mantenimiento": 120,
    "WO Vehicle Odometer": 45000,
    "Work Order Begin Date": "2024-05-10",
    "In Service Date": "2020-03-01"
}

X_prueba_time = preparar_input_tiempo(mi_input)
pred_time = model_time.predict(X_prueba_time)

print("\n===== RESULTADO DEL MODELO DE TIEMPO REAL =====")
print(f" Vehículo: {mi_input['Vehicle Make']} {mi_input['Vehicle Model']} {mi_input['Vehicle Year']}")
print(f" Tipo de mantenimiento: {mi_input['WO Reason']}")
print(f" Tiempo estimado real: {float(pred_time[0]):.2f} horas")



===== RESULTADO DEL MODELO DE TIEMPO REAL =====
 Vehículo: FORD F150 2019
 Tipo de mantenimiento: PREVENTIVE
 Tiempo estimado real: 3.88 horas


In [71]:
def preparar_input_ontime(datos): 
    """
    Prepara un registro para predecir el cumplimiento de tiempo (Y/N)
    usando las mismas transformaciones que se aplicaron a X_ontime.
    """

    df_in = pd.DataFrame([datos])

    # Asegurar columnas
    for col in X_ontime.columns:
        if col not in df_in.columns:
            df_in[col] = np.nan

    # Procesar categóricas igual que en entrenamiento
    for col in X_ontime.select_dtypes(include=['category']).columns:
        df_in[col] = df_in[col].astype(str).str.upper().str.strip()
        categorias = X_ontime[col].cat.categories

        df_in[col] = df_in[col].apply(
            lambda x: x if x in categorias else "UNKNOWN"
        )
        df_in[col] = pd.Categorical(df_in[col], categories=categorias)

    # Imputar numéricas
    for col in X_ontime.select_dtypes(include=['int64', 'float64']).columns:
        if df_in[col].isna().any():
            df_in[col] = df_in[col].fillna(X_ontime[col].median())

    return df_in[X_ontime.columns]


# =====================
#    EJEMPLO DE USO
# =====================
X_prueba_ot = preparar_input_ontime(mi_input)
pred_ot = model_on_time.predict(X_prueba_ot)

print("\n===== RESULTADO DEL MODELO DE CUMPLIMIENTO =====")
print(f" Vehículo: {mi_input['Vehicle Make']} {mi_input['Vehicle Model']}")
print(f" Cumplimiento esperado: {'Sí (Y)' if pred_ot[0] == 1 else 'No (N)'}")



===== RESULTADO DEL MODELO DE CUMPLIMIENTO =====
 Vehículo: FORD F150
 Cumplimiento esperado: Sí (Y)


In [75]:
def preparar_input_next(datos): #analizamos el modelo de proximo mantenimiento
    """
    Predecir días al próximo mantenimiento
    usando las mismas transformaciones aplicadas a X_next
    """

    df_in = pd.DataFrame([datos])

    # Asegurar columnas requeridas
    for col in X.columns:
        if col not in df_in.columns:
            df_in[col] = np.nan

    # --- CATEGÓRICAS ---
    for col in X.select_dtypes(include=['category']).columns:
        df_in[col] = df_in[col].astype(str).str.upper().str.strip()

        categorias = X[col].cat.categories
        df_in[col] = df_in[col].apply(lambda x: x if x in categorias else "UNKNOWN")

        df_in[col] = pd.Categorical(df_in[col], categories=categorias)

    # --- NUMÉRICAS ---
    for col in X.select_dtypes(include=['int64', 'float64']).columns:
        if df_in[col].isna().any():
            df_in[col] = df_in[col].fillna(X[col].median())

    return df_in[X.columns]


# =====================
#    EJEMPLO DE USO
# =====================
X_prueba_next = preparar_input_next(mi_input)
pred_next = model_next.predict(X_prueba_next)

print("\n===== RESULTADO DEL MODELO DE PRÓXIMO MANTENIMIENTO =====")
print(f" Vehículo: {mi_input['Vehicle Make']} {mi_input['Vehicle Model']}")
print(f" Días estimados hasta el próximo mantenimiento: {float(pred_next[0]):.1f} días")



===== RESULTADO DEL MODELO DE PRÓXIMO MANTENIMIENTO =====
 Vehículo: FORD F150
 Días estimados hasta el próximo mantenimiento: 99.1 días


In [77]:
def preparar_input(datos, modelo=""): #metodo general, llevarlo al ultimo con el resumen general
    """
    Prepara el input para predecir según el modelo solicitado.
    modelo = "cost", "time", "ontime", "next"
    """

    # ---- FEATURE SETS REALES DE TUS MODELOS ---- #
    cost_features = [
        'Vehicle Year', 'Vehicle Make', 'Vehicle Model', 'Vehicle Class',
        'Fuel Type', 'Division Owner', 'WO Reason', 'Priority Code',
        'Expected Hours', 'Shop Name', 'Dias_desde_ultimo_mantenimiento',
        'WO Vehicle Odometer', 'work_order_month', 'vehicle_age_years'
    ]

    time_features = [
        'Vehicle Make', 'Vehicle Model', 'Vehicle Class', 'WO Reason',
        'Priority Code', 'Expected Hours', 'Shop Name', 'work_order_month'
    ]

    ontime_features = [
        'Vehicle Make', 'Vehicle Model', 'Shop Name', 'Priority Code',
        'Expected Hours', 'work_order_month', 'work_order_dayofweek',
        'Division Owner', 'Vehicle Class'
    ]

    next_features = [
        'Dias_desde_ultimo_mantenimiento', 'WO Vehicle Odometer', 'Vehicle Year',
        'Vehicle Make', 'Vehicle Model', 'Vehicle Class', 'Fuel Type',
        'Division Owner', 'Expected Hours', 'WO Reason', 'Priority Code',
        'Shop Name', 'Days to Complete', 'Invoice Month Year', 'vehicle_age_years',
        'work_order_month', 'work_order_dayofweek', 'invoice_year'
    ]

    # Seleccionar columnas correctas
    if modelo == "cost":
        columnas = cost_features
    elif modelo == "time":
        columnas = time_features
    elif modelo == "ontime":
        columnas = ontime_features
    else:
        columnas = next_features

    # -----------------------------------------------------------------
    #       1. Crear DataFrame con TODAS las columnas necesarias
    # -----------------------------------------------------------------
    df = pd.DataFrame([{col: datos.get(col, None) for col in columnas}])

    # -----------------------------------------------------------------
    #       2. TIPOS DE DATOS
    # -----------------------------------------------------------------
    num_cols = df.select_dtypes(include=["int", "float"]).columns.tolist()
    cat_cols = df.select_dtypes(include=["object"]).columns.tolist()

    # Convertir numéricos
    for col in num_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Convertir categóricas (si el modelo espera category)
    for col in cat_cols:
        df[col] = df[col].astype(str).str.upper().str.strip()
        df[col] = df[col].replace({"NAN":"UNKNOWN", "NONE":"UNKNOWN", "": "UNKNOWN"})
        df[col] = df[col].astype("category")

        # Asegurar categoría UNKNOWN
        if "UNKNOWN" not in df[col].cat.categories:
            df[col] = df[col].cat.add_categories(["UNKNOWN"])

    # -----------------------------------------------------------------
    #       3. MANEJO DE FALTANTES
    # -----------------------------------------------------------------
    for col in df.columns:
        if df[col].isna().sum() > 0:
            if df[col].dtype.name == "category":
                df[col] = df[col].fillna("UNKNOWN")
            else:
                df[col] = df[col].fillna(0)

    return df


In [79]:
#se lo pasamos todo junto para predecir el costo
X_cost = preparar_input(mi_input, modelo="cost")
model_cost.predict(X_cost)

X_time = preparar_input(mi_input, modelo="time")
model_time.predict(X_time)

X_ontime = preparar_input(mi_input, modelo="ontime")
model_on_time.predict(X_ontime)

X_next = preparar_input(mi_input, modelo="next")
model_next.predict(X_next)

# xgboost nos devuelve un array de valor de predicción del mode
#lo (ya que le estamos pasando de un vehiculo)

array([445.53763], dtype=float32)

In [81]:
mi_input_2 = {
    "Vehicle Year": 2014,
    "Vehicle Make": "CHEVROLET",
    "Vehicle Model": "SILVERADO",
    "Vehicle Class": "H",
    "Fuel Type": "DIESEL",
    "Division Owner": "PUBLIC WORKS",
    "WO Reason": "REPAIR/MAINTENANCE",
    "Priority Code": "1",
    "Expected Hours": 6,
    "Shop Name": "HEAVY DUTY SHOP",
    "Dias_desde_ultimo_mantenimiento": 95,
    "WO Vehicle Odometer": 142000,
    "Work Order Begin Date": "2024-04-20",
    "In Service Date": "2014-05-01"
}

In [83]:
#Convertirlo a DataFrame
df_input = pd.DataFrame([mi_input_2])

In [85]:
#Convertir categóricas → category para xgboost para evitar errores de tipo
for col in cat_cols:  
    df_input[col] = df_input[col].astype(str).str.upper().astype("category")

In [89]:
pred = model_cost.predict(X_cost)[0] #analizamos el modelo de costo

print("===== RESULTADO DEL MODELO DE COSTO =====")
print(f" Vehículo: {mi_input_2['Vehicle Make']} {mi_input_2['Vehicle Model']} {mi_input_2['Vehicle Year']}")
print(f" Tipo de mantenimiento: {mi_input['WO Reason']}")
print(f" Costo estimado: ${pred:,.2f} USD")
print("=========================================")

===== RESULTADO DEL MODELO DE COSTO =====
 Vehículo: CHEVROLET SILVERADO 2014
 Tipo de mantenimiento: PREVENTIVE
 Costo estimado: $944.81 USD


In [91]:
def preparar_input_tiempo(datos): #analizamos el modelo de horas de trabajo
    """
    Predecir el tiempo real del mantenimiento (Actual Hours)
    basado en las mismas transformaciones que se aplicaron a X_time.
    """

    # 1. Convertir a DataFrame
    df_in = pd.DataFrame([datos])

    # 2. Asegurar que las columnas necesarias existen
    for col in X_time.columns:
        if col not in df_in.columns:
            df_in[col] = np.nan

    # 3. Procesar categóricas tal como se hizo en el entrenamiento
    for col in X_time.select_dtypes(include=['category']).columns:
        df_in[col] = df_in[col].astype(str).str.upper().str.strip()

        # Si no existe en las categorías del modelo, marcar como UNKNOWN
        categorias = X_time[col].cat.categories
        df_in[col] = df_in[col].apply(lambda x: x if x in categorias else 'UNKNOWN')

        df_in[col] = pd.Categorical(df_in[col], categories=categorias)

    # 4. Rellenar numéricas con mediana
    for col in X_time.select_dtypes(include=['int64','float64']).columns:
        if df_in[col].isna().any():
            df_in[col] = df_in[col].fillna(X_time[col].median())

    return df_in[X_time.columns]

# ==========================
#   EJEMPLO DE PREDICCIÓN
# ==========================

mi_input_2 = {
    "Vehicle Year": 2014,
    "Vehicle Make": "CHEVROLET",
    "Vehicle Model": "SILVERADO",
    "Vehicle Class": "H",
    "Fuel Type": "DIESEL",
    "Division Owner": "PUBLIC WORKS",
    "WO Reason": "REPAIR/MAINTENANCE",
    "Priority Code": "1",
    "Expected Hours": 6,
    "Shop Name": "HEAVY DUTY SHOP",
    "Dias_desde_ultimo_mantenimiento": 95,
    "WO Vehicle Odometer": 142000,
    "Work Order Begin Date": "2024-04-20",
    "In Service Date": "2014-05-01"
}

X_prueba_time = preparar_input_tiempo(mi_input)
pred_time = model_time.predict(X_prueba_time)

print("\n===== RESULTADO DEL MODELO DE TIEMPO REAL =====")
print(f" Vehículo: {mi_input_2['Vehicle Make']} {mi_input_2['Vehicle Model']} {mi_input_2['Vehicle Year']}")
print(f" Tipo de mantenimiento: {mi_input['WO Reason']}")
print(f" Tiempo estimado real: {float(pred_time[0]):.2f} horas")



===== RESULTADO DEL MODELO DE TIEMPO REAL =====
 Vehículo: CHEVROLET SILVERADO 2014
 Tipo de mantenimiento: PREVENTIVE
 Tiempo estimado real: 4.16 horas
